In [197]:
using Revise, DynamicHMC, PosteriorDB, Random, StanLogDensityProblems, JSON, BridgeStan, DataFrames, LinearAlgebra, StatsBase, Distributions
using Plots, ColorSchemes, LaTeXStrings, Statistics, CSV, AdvancedHMC, DynamicObjects
Plots.theme(:default)
const BS = BridgeStan;

In [198]:
pdb = PosteriorDB.database()
model = PosteriorDB.model(pdb, "eight_schools_noncentered")
post = PosteriorDB.posterior(pdb, "eight_schools-eight_schools_noncentered")
impl = PosteriorDB.implementation(model, "stan")
path = PosteriorDB.path(impl)
data = PosteriorDB.dataset(post)
s = PosteriorDB.load(data, String);
bs_model = BS.StanModel(stan_file=path, data=s);
ref = PosteriorDB.reference_posterior(post)
df=DataFrame(PosteriorDB.load(ref));

┌ Warning: Loading a shared object '/home/meenaljhajharia/.julia/artifacts/95c3074bd791a2f28857088639b48ca1b786459a/posteriordb-0.4.0/posterior_database/models/stan/eight_schools_noncentered_model.so' which is already loaded.
│ If the file has changed since the last time it was loaded, this load may not update the library!
└ @ BridgeStan /home/meenaljhajharia/.julia/packages/BridgeStan/nCPwJ/src/model.jl:51


In [199]:
using TransformVariables
using TransformedLogDensities, LogDensityProblemsAD
t = as((μ = asℝ, σ = asℝ₊, τ = asℝ₊, θs = as(Array, 8)))
dimension(t)
P = TransformedLogDensity(t, lp)
∇P = ADgradient(:ForwardDiff, lp);

p = StanProblem(bs_model);
D = 10;
initial_θ = rand(D);
n_samples, n_adapts = 8000, 2000;
metric = DiagEuclideanMetric(D);
hamiltonian = Hamiltonian(metric, lp);
initial_ϵ = find_good_stepsize(hamiltonian, initial_θ);
integrator = Leapfrog(initial_ϵ);
proposal = NUTS{MultinomialTS, GeneralisedNoUTurn}(integrator);
adaptor = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(0.8, integrator));
samples, stats = sample(hamiltonian, proposal, initial_θ, n_samples, adaptor, n_adapts; progress=true);

Sampling  26%|████████▏                      |  ETA: 0:00:00
  iterations:                                   2096
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.01
  n_steps:                                      15
  is_accept:                                    true
  acceptance_rate:                              0.9198919727715424
  log_density:                                  -10.151961486450443
  hamiltonian_energy:                           14.866417664971074
  hamiltonian_energy_error:                     0.0639968462763818
  max_hamiltonian_energy_error:                 11.208003626296176
  tree_depth:                                   3
  numerical_error:                              false
  step_size:                                    0.43240622477448887
  nom_step_size:                                0.43240622477448887
  is_adapt:                                     false



















Sampling  53%|████████████████▌              |  ETA: 0:00:00
  iterations:                                   4261
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.01
  n_steps:                                      7
  is_accept:                                    true
  acceptance_rate:                              1.0
  log_density:                                  -5.36568105333099
  hamiltonian_energy:                           7.8911350824160476
  hamiltonian_energy_error:                     -0.034997733525740315
  max_hamiltonian_energy_error:                 -0.15040237936167333
  tree_depth:                                   3
  numerical_error:                              false
  step_size:                                    0.43240622477448887
  nom_step_size:                                0.43240622477448887
  is_adapt:                                     false



















Sampling  89%|███████████████████████████▋   |  ETA: 0:00:00
  iterations:                                   7132
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.0
  n_steps:                                      7
  is_accept:                                    true
  acceptance_rate:                              1.0
  log_density:                                  -7.8808852099672535
  hamiltonian_energy:                           12.886839126913582
  hamiltonian_energy_error:                     -0.1293309464772392
  max_hamiltonian_energy_error:                 -0.28522378246468705
  tree_depth:                                   3
  numerical_error:                              false
  step_size:                                    0.43240622477448887
  nom_step_size:                                0.43240622477448887
  is_adapt:                                     false



















Sampling 100%|███████████████████████████████| Time: 0:00:00
  iterations:                                   8000
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.0
  n_steps:                                      7
  is_accept:                                    true
  acceptance_rate:                              0.9862576804234344
  log_density:                                  -6.782090218875673
  hamiltonian_energy:                           11.47621845809056
  hamiltonian_energy_error:                     -0.004317515155708662
  max_hamiltonian_energy_error:                 -0.06203163602238959
  tree_depth:                                   3
  numerical_error:                              false
  step_size:                                    0.43240622477448887
  nom_step_size:                                0.43240622477448887
  is_adapt:                                     false
  mass_matrix:               

┌ Info: Finished 8000 sampling steps for 1 chains in 0.358179706 (s)
│   h = Hamiltonian(metric=DiagEuclideanMetric([0.8881970378388442, 0.9325 ...]), kinetic=GaussianKinetic())
│   κ = HMCKernel{AdvancedHMC.FullMomentumRefreshment, Trajectory{MultinomialTS, Leapfrog{Float64}, GeneralisedNoUTurn{Float64}}}(AdvancedHMC.FullMomentumRefreshment(), Trajectory{MultinomialTS}(integrator=Leapfrog(ϵ=0.432), tc=GeneralisedNoUTurn{Float64}(10, 1000.0)))
│   EBFMI_est = 1.0219386574304055
│   average_acceptance_rate = 0.8606366511701165
└ @ AdvancedHMC /home/meenaljhajharia/.julia/packages/AdvancedHMC/P0nla/src/sampler.jl:246


In [200]:
constrained_draws = hcat([vcat(col...) for col in eachcol(df)]...)
unc_sample_array = vcat([
    param_unconstrain(bs_model, collect(row))' for row in eachrow(constrained_draws)
]...);

In [201]:
lol = hcat(samples...)'
conhmc = vcat([
    param_constrain(bs_model, collect(row))' for row in eachrow(lol)
]...);

In [202]:
mean(unc_sample_array, dims=1)

1×10 Matrix{Float64}:
 6.1505  4.93958  3.90591  4.79602  …  6.31717  4.884  4.41052  0.808081

In [203]:
mean(conhmc, dims=1)

1×10 Matrix{Float64}:
 0.326793  0.102662  -0.0927928  0.0482775  …  0.071242  4.40177  3.60675